## Подготовка - установка библиотек

In [ ]:
# Установка библиотек

!pip install transformers torch sentencepiece nltk rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

## Подготовка данных и модели

In [ ]:
# Загрузка датасета

import pandas as pd
df = pd.read_csv('lenta.csv', index_col=None)
df['Description'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)
df = df[['Text', 'Description']]

In [ ]:
# Разделение на обучающую и тестовую выборки

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Уменьшение использования памяти GPU
# torch.cuda.set_per_process_memory_fraction(0.5, max_split_size_mb=200)

In [ ]:
# Загрузка предобученной модели и токенизатора

# model_name = "ai-forever/ruT5-base"
model_name = "sergeonsr/ruT5-base-test"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

(…)-test/resolve/main/tokenizer_config.json:   0%|          | 0.00/20.5k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

(…)base-test/resolve/main/added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

(…)est/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(…)/ruT5-base-test/resolve/main/config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

(…)test/resolve/main/generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## Обучение

In [ ]:
# Предварительная обработка данных
def preprocess_data(texts, max_length=1024):
    inputs = tokenizer(texts, max_length=max_length, return_tensors='pt', truncation=True, padding='max_length')
    return inputs

In [ ]:
# Обучение модели
def train_model(inputs, labels, model, tokenizer, batch_size=1, num_epochs=4, learning_rate=5e-5):
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    input_ids = inputs['input_ids']
    target_ids = labels['input_ids']

    dataset_size = len(input_ids)
    num_batches = (dataset_size + batch_size - 1) // batch_size

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = (batch_idx + 1) * batch_size

            input_batch = input_ids[start_idx:end_idx]
            target_batch = target_ids[start_idx:end_idx]

            # Передача батча данных, рассчет потерь
            outputs = model(input_ids=input_batch, labels=target_batch)
            loss = outputs.loss
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_loss = total_loss / num_batches
        print(f'Epoch {epoch + 1}, Loss: {average_loss}')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Перемещение данных на то же устройство, что и модель
input_texts = train_df['Text'].tolist()
target_texts = train_df['Description'].tolist()
# Преобразование текстов в тензоры
inputs = tokenizer(input_texts, return_tensors='pt', truncation=True, padding=True)
labels = tokenizer(target_texts, return_tensors='pt', truncation=True, padding=True)
# Перемещение на устройство
inputs = {key: value.to(device) for key, value in inputs.items()}
labels = {key: value.to(device) for key, value in labels.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Обучение модели
train_model(inputs, labels, model, tokenizer, batch_size=1, num_epochs=4)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.49077616140140956
Epoch 2, Loss: 0.2209935302011458
Epoch 3, Loss: 0.12296265338740821
Epoch 4, Loss: 0.11390756318666358


## Инференс

In [ ]:
def generate_summary(text, model, tokenizer, max_length=200):
    # Преобразование текста в тензор и перемещение на устройство модели
    inputs = tokenizer(text, max_length=max_length, return_tensors='pt', truncation=True)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Генерация суммаризации
    summary_ids = model.generate(**inputs, max_length=200, min_length=50, length_penalty=2.0, num_beams=4)

    # Преобразование результата в строку
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

Сильная магнитная буря вызвала редкое полярное сияние на юге России. Красное свечение могли увидеть жители Крыма, Ростовской и Воронежской областей, а также новых регионов страны, передает РБК.  Астроном Александр Иванов в беседе с «Комсомольской Правдой» объяснил, что это явление является очень редким для южных регионов. Полярное сияние длится от пяти минут до получаса и происходит однократно.  «Полярное сияние возникает от протонных ударов по магнитосфере. Магнитные бури возникли недавно, с обеда 5 ноября. Продолжаются три часа подряд. Свечение как раз связано с этим. Фактически северное сияние, механизм такой же», — отметил он.  В конце сентября в России засняли редкое по красоте полярное сияние. В ночь на 25 сентября жители Советска могли увидеть в небе над рекой Неман полярное сияние в ярком зеленом и алом спектре.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
# Пример использования
sample_text = """
Сильная магнитная буря вызвала редкое полярное сияние на юге России. Красное свечение могли увидеть жители Крыма, Ростовской и Воронежской областей, а также новых регионов страны, передает РБК.  Астроном Александр Иванов в беседе с «Комсомольской Правдой» объяснил, что это явление является очень редким для южных регионов. Полярное сияние длится от пяти минут до получаса и происходит однократно.  «Полярное сияние возникает от протонных ударов по магнитосфере. Магнитные бури возникли недавно, с обеда 5 ноября. Продолжаются три часа подряд. Свечение как раз связано с этим. Фактически северное сияние, механизм такой же», — отметил он.  В конце сентября в России засняли редкое по красоте полярное сияние. В ночь на 25 сентября жители Советска могли увидеть в небе над рекой Неман полярное сияние в ярком зеленом и алом спектре."
"""
summary = generate_summary(sample_text, model, tokenizer)
print("Суммаризация:")
print(summary)

Суммаризация:
Сильная магнитная буря вызвала редкое полярное сияние на юге России. Красное свечение могли увидеть жители Крыма, Ростовской и Воронежской областей, а также новых регионов страны. Об этом сообщил «Комсомольская Правда». Астроном Александр Иванов объяснил, что это явление является очень редким для южных регионов.


## Сохранение модели и токенизатора

In [ ]:
# Сохранение модели и токенизатора
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/spiece.model',
 'saved_model/added_tokens.json')

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer.push_to_hub("ruT5-base-test")

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sergeonsr/ruT5-base-test/commit/81b03c7b99fdbe8ec73badc2c95776ffd987d27f', commit_message='Upload tokenizer', commit_description='', oid='81b03c7b99fdbe8ec73badc2c95776ffd987d27f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("ruT5-base-test")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sergeonsr/ruT5-base-test/commit/895d8b935d2549bf4981af591a31e42524f241f4', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='895d8b935d2549bf4981af591a31e42524f241f4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Оценка модели с использованием метрик BLEU и ROUGE
def generate_summary(text):
    input_ids = tokenizer(text, return_tensors="pt", max_length=max_input_length, truncation=True).input_ids.to('cuda')
    summary_ids = model.generate(input_ids, max_length=max_output_length, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
generated_summaries = [generate_summary(text) for text in test_df['Text'].tolist()]

In [ ]:
generated_summaries

[' Дочь Валентина унаменитого отца два моды — в Вознесеском переформатировать превое здание, и западе столицы, на Кутузов',
 ' В Ленинградской области расчлененного мужчину нашли в реке Перовка, проходящей рядом с Скандинавия». Фрагменты тела н',
 ' В Яндекс Картах» с 19 октября автомобилисты могут увидеть стоимость по платным дорогам. Такую функцию приложении на показывается цена',
 ' В Екатеринбурге произошел крупный пожар на строящейся Ледовой арене. По данным Telegram-канала Ural Mash. К месту возгорания н',
 ' Красноярский гарнизонный суд приговорил рядового Семенова к пяти годам лишения свободы за самов',
 ' Норвегия попросили испражняться в пластиковые пакеты, что поэтому проблему обеловеческих фекалий островах. Уточняется, люд',
 ' Президент России Владимир Путин объявил о начале постоянного патрулирования Воздушно-космическим силами (ВКС) Рсской Федерации  прос',
 ' Американец Даниэль Кастл приехал в Россию в 2022 году, чтобы увидеть Сибирь и Байкал, и «�стался в стран',
 ' В 

In [ ]:
# Метрика BLEU

references = [[summary] for summary in test_df['Description']]
bleu_score = corpus_bleu(references, generated_summaries)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.22245449284702293


In [ ]:
# Метрика ROUGE

rouge = Rouge()
scores = rouge.get_scores(generated_summaries, test_df['Description'].tolist(), avg=True)
print(scores)

{'rouge-1': {'r': 0.3126676380567927, 'p': 0.6935416219964441, 'f': 0.4221393539143733}, 'rouge-2': {'r': 0.22311026978925946, 'p': 0.5372351461970604, 'f': 0.30765032900535044}, 'rouge-l': {'r': 0.3074505141214003, 'p': 0.6821857499862272, 'f': 0.41504256239551784}}


In [ ]:
# Пример текста для суммирования
example_text = """
В Паттайе полиция задержала и отправила в больницу российского туриста, ходившего по улице голым и нападавшего на окружающих, пишет местная газета The Thaiger. По данным издания, 23-летний гражданин России ходил по городу полностью обнаженным и вел себя агрессивно. Как рассказали очевидцы, он кричал на прохожих и ударил водителя мотоцикла, стоявшего на перекрестке. Сообщается, что при задержании турист продолжал угрожать полиции и репортерам, а позже и медикам в больнице. Там мужчине проведут психиатрическую экспертизу и проверят кровь на наличие запрещенных препаратов, после чего ему предъявят обвинение
"""

# Токенизация и генерация суммаризации
input_ids = tokenizer(example_text, return_tensors="pt", truncation=True).input_ids.to('cuda')
summary_ids = model.generate(input_ids, max_length=256, num_beams=4, length_penalty=2.0, early_stopping=False)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Вывод результатов
print("Исходный текст:")
print(example_text)
print("\nСгенерированная суммаризация:")
print(summary)

Исходный текст:

В Паттайе полиция задержала и отправила в больницу российского туриста, ходившего по улице голым и нападавшего на окружающих, пишет местная газета The Thaiger. По данным издания, 23-летний гражданин России ходил по городу полностью обнаженным и вел себя агрессивно. Как рассказали очевидцы, он кричал на прохожих и ударил водителя мотоцикла, стоявшего на перекрестке. Сообщается, что при задержании турист продолжал угрожать полиции и репортерам, а позже и медикам в больнице. Там мужчине проведут психиатрическую экспертизу и проверят кровь на наличие запрещенных препаратов, после чего ему предъявят обвинение


Сгенерированная суммаризация:
 В Паттайе полиция задержала и отправила российского туриста, ходившего нападавющих, он кричал в больностью обнаженным. Как рассказали очевидцы, по городу и ударил себя агрессно. Сообщается при з
